### Sliding Window Attention


In [5]:
import torch
import torch.nn as nn
import math
import torch.nn.functional as F


In [17]:
input = torch.randn(1, 5, 5)
input

tensor([[[ 0.8229,  0.8025, -0.5787, -0.3725,  0.2452],
         [ 0.0366, -0.3871,  1.5102,  0.9443, -0.1497],
         [-0.6076, -0.5786, -0.4912,  2.0292, -0.0453],
         [-0.3350,  0.6715, -0.6786, -0.2576, -0.3165],
         [ 0.6325, -0.1481,  0.4003,  1.5457,  0.2917]]])

In [33]:
    
def causal_mask(size):
    mask = torch.triu(torch.ones((1, size, size)), diagonal=1).type(torch.int)
    return mask == 0

def sliding_mask(size):
    mask = torch.triu(torch.ones((1, size, size)), diagonal=-2).type(torch.int)
    return mask == 1

#for a sliding window of size 3, diagonal = -2, size 4 diagonal -3 etc

In [34]:
causal_mask(8).int()

tensor([[[1, 0, 0, 0, 0, 0, 0, 0],
         [1, 1, 0, 0, 0, 0, 0, 0],
         [1, 1, 1, 0, 0, 0, 0, 0],
         [1, 1, 1, 1, 0, 0, 0, 0],
         [1, 1, 1, 1, 1, 0, 0, 0],
         [1, 1, 1, 1, 1, 1, 0, 0],
         [1, 1, 1, 1, 1, 1, 1, 0],
         [1, 1, 1, 1, 1, 1, 1, 1]]], dtype=torch.int32)

In [35]:
sliding_mask(8).int()

tensor([[[1, 1, 1, 1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1, 1, 1, 1],
         [0, 1, 1, 1, 1, 1, 1, 1],
         [0, 0, 1, 1, 1, 1, 1, 1],
         [0, 0, 0, 1, 1, 1, 1, 1],
         [0, 0, 0, 0, 1, 1, 1, 1],
         [0, 0, 0, 0, 0, 1, 1, 1]]], dtype=torch.int32)

In [36]:
(causal_mask(8) & sliding_mask(8)).int()

tensor([[[1, 0, 0, 0, 0, 0, 0, 0],
         [1, 1, 0, 0, 0, 0, 0, 0],
         [1, 1, 1, 0, 0, 0, 0, 0],
         [0, 1, 1, 1, 0, 0, 0, 0],
         [0, 0, 1, 1, 1, 0, 0, 0],
         [0, 0, 0, 1, 1, 1, 0, 0],
         [0, 0, 0, 0, 1, 1, 1, 0],
         [0, 0, 0, 0, 0, 1, 1, 1]]], dtype=torch.int32)

### RMSNorm

In [ ]:

class RMSNormalization(nn.Module):

    def __init__(self, eps:float=10**-8) -> None:
        super().__init__()
        self.eps = eps
        self.alpha = nn.Parameter(torch.ones(1)) # alpha is a learnable parameter
        # self.bias = nn.Parameter(torch.zeros(1)) # bias is a learnable parameter

    def forward(self, x):

        # x: (batch, seq_len, hidden_size)
         # Keep the dimension for broadcasting
        squared_x = torch.square(x) 
        mean = squared_x.mean(dim = -1, keepdim = True) # (batch, seq_len, 1)
        
        # Keep the dimension for broadcasting
        sqrt_mean = torch.sqrt(mean) # (batch, seq_len, 1) 


        # eps is to prevent dividing by zero or when sqrt_mean is very small
        return self.alpha * (x ) / (sqrt_mean + self.eps)

### Rotatery Embedding

In [ ]:
# formula for theta is 10000 ^ -2(i-1)/d
# i [ real numbers from 1 to d/2]
# we can write it as 1/10000 ^ 2([0 - 256]/d)

class RotateryEmbedding(nn.Module):

    def __init__(self,seq_len:int,  constant = 10000.0 ,embedding_dim:int=512) -> None:
        super().__init__()
        self.eps = eps
        self.theta = theta
        self.aimport torch

# Assuming you have two tensors (replace these with your actual data)
matrix1 = torch.tensor([1, 2, 3])
matrix2 = torch.tensor([4, 5, 6])


torch.outer(matrix1, matrix2)lpha = nn.Parameter(torch.ones(1)) # alpha is a learnable parameter
        # self.bias = nn.Parameter(torch.zeros(1)) # bias is a learnable parameter

    def pre_compute_theta(self, x):


        i = torch.arange(0, 256).float()

        theta = 1.0 /(constant **((2 * i)/ embedding_dim) )

        m = torch.arange(seq_len)

        freq = torch.outer(m, theta).float()

        # reps the freq in complex form cos(m*theta) + i sin(m*theta)

        freqs_complex_form = torch.polar(torch.ones_like(freq), freq)

        return freqs_complex_form
    def apply_rotary_pos_encoding( x: torch.Tensor, freqs_complex_form: torch.Tensor):  
        x_complex = torch.view_as_complex(x.float().reshape(*x.shape[:-1], -1))

        freqs_complex_form = freqs_complex_form.unsqueeze(0).unsqueeze(2) 

        x_rotated = x_complex * freqs_complex_form

        x_out = torch.view_as_real(x_rotated) 

        return x_out.reshape(*x.shape).type_as(x).to(device)
        # # x: (batch, seq_len, hidden_size)
        #  # Keep the dimension for broadcasting
        # squared_x = torch.square(x) 
        # mean = squared_x.mean(dim = -1, keepdim = True) # (batch, seq_len, 1)
        
        # # Keep the dimension for broadcasting
        # sqrt_mean = torch.sqrt(mean) # (batch, seq_len, 1) 


        # # eps is to prevent dividing by zero or when sqrt_mean is very small
        # return self.alpha * (x ) / (sqrt_mean + self.eps)


### KV_ CACHE


In [42]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model:int, heads: int) -> None:
        super(MultiHeadAttention,self).__init__()
        self.head = heads
        self.head_dim = d_model // heads
        self.d_model = d_model
        


        assert d_model % heads == 0, 'cannot divide d_model by heads'

        ## initialize the query, key and value weights 512*512
        self.query_weight = nn.Linear(d_model, d_model, bias=False)
        self.key_weight = nn.Linear(d_model, d_model,bias=False)
        self.value_weight = nn.Linear(d_model, d_model,bias=False)
        self.final_weight  = nn.Linear(d_model, d_model, bias=False)
        self.dropout = nn.Dropout(p=0.1)

        ## initialize the query, key and value matrices to give us batch, seq_len by 512
        self.key_cache = torch.empty(1,0,d_model)
        self.value_cache = torch.empty(1,0,d_model)

      
    def self_attention(self,query, key, value, mask,dropout):

        print(f"key cache {self.key_cache.shape}")
        print(f"value cache {self.value_cache.shape}")

        # rolling buffer ???? 
        if self.key_cache.shape == torch.Size([1,4,self.d_model]):
            print(f'actual_matrix {self.key_cache}')
            self.key_cache = torch.cat((self.key_cache[:, 1:,:], key), dim=1)
            self.value_cache = torch.cat((self.value_cache[:, 1:,:], value), dim=1)

            print(f'actual_matrix_after {self.key_cache}')
        else:    
   
            self.key_cache = torch.cat((self.key_cache, key), dim=1)
            self.value_cache = torch.cat((self.value_cache, value), dim=1)



             #splitting query, key and value into heads
                #this gives us a dimension of batch, num_heads, seq_len by 64. basically 1 sentence is converted to have 8 parts (heads)
        print(f"key cache_ {self.key_cache.shape}")
        print(f"value cache_ {self.value_cache.shape}")        
        query = query.view(query.shape[0], query.shape[1],self.head,self.head_dim).transpose(2,1)
        key = self.key_cache.view(self.key_cache.shape[0], self.key_cache.shape[1],self.head,self.head_dim).transpose(2,1)
        value = self.value_cache.view(self.value_cache.shape[0], self.value_cache.shape[1],self.head,self.head_dim).transpose(2,1)
        
        attention = query @ key.transpose(3,2)
        attention = attention / math.sqrt(query.shape[-1])

        if mask is not None:
           attention = attention.masked_fill(mask == 0, -1e9)      
        attention = torch.softmax(attention, dim=-1)      
        if dropout is not None:
            attention = dropout(attention)
        attention_scores =  attention @ value    
       
        return attention_scores.transpose(2,1).contiguous().view(attention_scores.shape[0], -1, self.head_dim * self.head)
      
    def forward(self,query, key, value,mask):

        ## initialize the query, key and value matrices to give us seq_len by 512
        query = self.query_weight(query)
        key = self.key_weight(key)
        value = self.value_weight(value)

        attention = MultiHeadAttention.self_attention(self, query, key, value, mask, self.dropout)
        return self.final_weight(attention) 



In [43]:
mult = MultiHeadAttention(6,2)
# MultiHeadAttention(4, 2)(torch.randn(1,1,4), torch.randn(1,1,4), torch.randn(1,1,4), None)

In [50]:
mult(torch.randn(1,1,6), torch.randn(1,1,6), torch.randn(1,1,6), None)

key cache torch.Size([1, 4, 6])
value cache torch.Size([1, 4, 6])
actual_matrix tensor([[[-0.3789,  0.0984, -0.5503,  0.0935, -0.9363,  0.3123],
         [ 0.0748, -0.0306,  0.1323, -1.4742, -0.0179,  1.7810],
         [-0.6320,  0.7551,  0.0666, -0.4027, -1.3768,  0.4777],
         [ 0.4627,  0.4404,  0.2701,  0.1492, -0.3385,  0.5148]]],
       grad_fn=<CatBackward0>)
actual_matrix_after tensor([[[ 0.0748, -0.0306,  0.1323, -1.4742, -0.0179,  1.7810],
         [-0.6320,  0.7551,  0.0666, -0.4027, -1.3768,  0.4777],
         [ 0.4627,  0.4404,  0.2701,  0.1492, -0.3385,  0.5148],
         [-0.6304,  0.0575, -0.3107, -0.2914, -0.5584,  0.1118]]],
       grad_fn=<CatBackward0>)
key cache_ torch.Size([1, 4, 6])
value cache_ torch.Size([1, 4, 6])


tensor([[[ 0.0941, -0.0206,  0.0677, -0.0313, -0.0363,  0.1189]]],
       grad_fn=<UnsafeViewBackward0>)

In [22]:
x = torch.randn(1,1,2,3)
x

tensor([[[[-1.3062,  0.5340,  0.6917],
          [ 0.5460, -0.1276,  0.1350]]]])

In [23]:
x.transpose(2,1)

tensor([[[[-1.3062,  0.5340,  0.6917]],

         [[ 0.5460, -0.1276,  0.1350]]]])

In [144]:
x = torch.cat((x,torch.randn(1,1,1)), dim=1)
x.shape

torch.Size([1, 12, 1])